In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
/;


....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


In [3]:
sub get_caret {
    my ($input) = @_;

    (grep defined,
        flatten_nd
        map_nd selector('where.coord'),
        map_nd_indexed graphql_query("{ where(value:'^') { coord }}"),
        parse_2d_map_array($input))[0];
}

In [4]:
$graphql::graphql_methods{row} = sub {
        my ($self, $params) = @_;
        my @row = @{$self->{arr}[$self->{coord}[0]]};
        return [ @row ];
    };
$graphql::graphql_methods{column} = sub {
        my ($self, $params) = @_;
        my ($a, $b) = @{$self->{coord}};
        my @column = map $_->[$b], @{$self->{arr}};
        return \@column;
    };

my @pos_table = 'a' .. 'z';

$graphql::graphql_methods{row_slice} = sub {
        my ($self, $params) = @_;
        my $new_coord = [
            map { $params->{"$pos_table[$_]"} // ($self->{coord}[$_] + ($params->{"d$pos_table[$_]"} // 0)) } 0 .. $#{$self->{coord}} ];
        my ($a, $b) = @$new_coord;
        my ($from, $to) = @{$params}{'from', 'to'};
        $from //= $b;
        $to //= $b;
        
        my @row = @{$self->{arr}[$self->{coord}[0]]};
        $to = $to eq 'n' ? $#row : $to;
        $to = $to =~ /^x(([+\-]\d+))?$/ ? $b+$1 : $to;
        $from = $from =~ /^x(([+\-]\d+))?$/ ? $b+$1 : $from;
        return [ @row[ $from .. $to ] ];
    };

sub shape_nd {
    my ($arr) = @_;
	return [ scalar @$arr ] if (@$arr >= 0 and ref $arr->[0] ne 'ARRAY');
	return [ scalar (@$arr), @{shape_nd($arr->[0])} ];
}

sub group_by {
    my ($key, @data) = @_;
    my %table;
    foreach (@data) {
        push @{$table{$_->{$key}}}, $_;
    }
    return %table;
}

sub group {
    my (@data) = @_;
    my %table;
    foreach (@data) {
        push @{$table{$_}}, $_;
    }
    return %table;
}


CODE(0x55fdc9627f98)

In [5]:

sub point_range {
    my ($a, $b) = @_;
    my $min_a = $a->[0] < $b->[0] ? $a->[0] : $b->[0];
    my $max_a = $a->[0] > $b->[0] ? $a->[0] : $b->[0];
    my $min_b = $a->[1] < $b->[1] ? $a->[1] : $b->[1];
    my $max_b = $a->[1] > $b->[1] ? $a->[1] : $b->[1];

    return map { my $n = $_; map { [$n, $_] } $min_b .. $max_b } $min_a .. $max_a;
}

sub process_day6_part1_get_stepped {
    my ($input) = @_;

    my $caret = get_caret($input);
    my @points =
        grep defined,
        flatten_nd
        map_nd selector(qq/where.pos/),
        map_nd_indexed graphql_query(qq/{ where(value:'#') {
            pos
        }}/),
        parse_2d_map_array($input);
    my %point_table = map { ($_->{a} . ',' . $_->{b}) => 1 } @points;
    my $shape = shape_nd(parse_2d_map_array($input));

    my %v_table = group_by('b', @points);
    my %h_table = group_by('a', @points);

    my $direction = [-1,0];

    my $next = 1;
    my $to_caret;
    my %stepped_points;

    while ($next) {
        # calculate collisions
        if ($direction->[0] == 0) {
            $next = (
                sort { abs($a->{b} - $caret->[1]) <=> abs($b->{b} - $caret->[1]) }
                grep $_->{b} * -$direction->[1] < $caret->[1] * -$direction->[1],
                @{$h_table{$caret->[0]}})[0];
        } else {
            $next = (
                sort { abs($a->{a} - $caret->[0]) <=> abs($b->{a} - $caret->[0]) }
                grep $_->{a} * -$direction->[0] < $caret->[0] * -$direction->[0],
                @{$v_table{$caret->[1]}})[0];
        }

        last unless defined $next;

        # calculate next caret and the points that we crossed
        $to_caret = [ $next->{a} - $direction->[0], $next->{b} - $direction->[1] ];
        @stepped_points{map { join ',', @$_ } point_range($caret, $to_caret)} = ();
        warn "break on moving from $caret to $to_caret" if (grep exists $point_table{$_}, map { join ',', @$_ } point_range($caret, $to_caret));

        # update state
        $caret = $to_caret;
        $direction = [ $direction->[1], -$direction->[0] ];
    }

    # no further collisions, we go all the way to the wall
    $to_caret = [
        ($caret->[0], $shape->[0]-1, 0)[$direction->[0]],
        ($caret->[1], $shape->[1]-1, 0)[$direction->[1]],
    ];
    @stepped_points{map { join ',', @$_ } point_range($caret, $to_caret)} = ();
    warn "break on moving from $caret to $to_caret" if (grep exists $point_table{$_}, map { join ',', @$_ } point_range($caret, $to_caret));
    # \%point_table;
    keys %stepped_points;
    # sum map 1, keys %stepped_points;
    # $shape;
    # $to_caret;
}

sub process_day6_part1 {
    sum map 1, process_day6_part1_get_stepped(@_);
}
# 0 < 4
# 7 < 4

# 0*-1 < 4*-1 : 0 < -4
# 7*-1 < 4*-1 : -7 < -4

# say Dumper get_caret($practice_input);
say Dumper process_day6_part1($practice_input);

$VAR1 = 41;



1

In [6]:

sub update_map_path {
    my ($map, $shape, $path) = @_;
    foreach my $point (map [ split ',' ], @$path) {
        # say 'wat:', $map->[$point->[0] * $shape->[1] + $point->[1]];
        warn 'assertion failed at ', $point->[0] . ',' . $point->[1] if $map->[$point->[0] * $shape->[1] + $point->[1]] eq '#';
        $map->[$point->[0] * $shape->[1] + $point->[1]] = $map->[$point->[0] * $shape->[1] + $point->[1]] eq '#' ? '!' : 'X';
    }
}
sub string_map {
    my ($map, $shape) = @_;
    join "\n", map { join '', @{$map}[$_ * $shape->[1] .. (($_+1) * $shape->[1]-1)] } 0 .. $shape->[0] - 1;
}
my $data = parse_2d_map_array($practice_input);
my $shape = shape_nd($data);
my @map = flatten_nd $data;
update_map_path(\@map, $shape, [ process_day6_part1_get_stepped($practice_input) ]);
say string_map(\@map, $shape);



....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..


1

In [7]:
my $input = get_challenge('2024/day/6/input');
# my $res = process_day6_part1($input);

my $data = parse_2d_map_array($input);
my $shape = shape_nd($data);
my @map = flatten_nd $data;
update_map_path(\@map, $shape, [ process_day6_part1_get_stepped($input) ]);
say string_map(\@map, $shape);
writefile('.exp/testmap', string_map(\@map, $shape));

...#....X...................................#........#.................#............#.#.......................#...............#..#
...#....X...#.....#..................................................................................#......#..#..................
........X........#.......................#.#........#......#...........................................#...#..........#...........
........X.........#..................#...#.....................................#........#.........................................
...#....X#.......#..............#.#....##.............#....##......................................#...................#..........
...#....XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#...
..#..#..XX.........#.....#...............#.......#..............................................#....#.......................X....
........XX.........XXX#..#...............#...................#....#................

1

In [8]:
my $input = get_challenge('2024/day/6/input');
my $res = process_day6_part1($input);

5516

In [9]:
# say post_answer('2024/day/6/answer', 1, $res);

In [10]:

sub get_next_collision {
    my ($position, $direction, $h_table, $v_table) = @_;
    # calculate collisions
    if ($direction->[0] == 0) {
        return (
            sort { abs($a->{b} - $position->[1]) <=> abs($b->{b} - $position->[1]) }
            grep $_->{b} * -$direction->[1] < $position->[1] * -$direction->[1],
            @{$h_table->{$position->[0]}})[0];
    } else {
        return (
            sort { abs($a->{a} - $position->[0]) <=> abs($b->{a} - $position->[0]) }
            grep $_->{a} * -$direction->[0] < $position->[0] * -$direction->[0],
            @{$v_table->{$position->[1]}})[0];
    }
}

sub test_find_loop {
    my ($from, $to, $direction, $h_table, $v_table, $previous_impacts) = @_;
    my $next_direction = [ $direction->[1], -$direction->[0] ];
    my @loops_found;
    foreach my $point (point_range($from, $to)) {
        my $test_collision = get_next_collision($point, $next_direction, $h_table, $v_table);
        next unless defined $test_collision;
        # say "found wat:", Dumper $test_collision;
        my @matches = grep {
            $test_collision->{a} == $_->[0]{a}
            && $test_collision->{b} == $_->[0]{b}
            && $next_direction->[0] == $_->[1][0]
            && $next_direction->[1] == $_->[1][1]} @$previous_impacts;
        if (@matches > 0) {
            # say "found match:", Dumper \@matches;
            push @loops_found, $matches[0];
        }
    }
    return @loops_found;
}

sub process_day6_part2_get_stepped {
    my ($input) = @_;

    my $caret = get_caret($input);
    my @points =
        grep defined,
        flatten_nd
        map_nd selector(qq/where.pos/),
        map_nd_indexed graphql_query(qq/{ where(value:'#') {
            pos
        }}/),
        parse_2d_map_array($input);
    my %point_table = map { ($_->{a} . ',' . $_->{b}) => 1 } @points;
    my $shape = shape_nd(parse_2d_map_array($input));

    my %v_table = group_by('b', @points);
    my %h_table = group_by('a', @points);

    my $direction = [-1,0];

    my $next = 1;
    my $to_caret;
    my @stepped_points;
    my $last_direction;

    my @previous_impacts;
    my @loops_found;

    while ($next) {
        $last_direction = $direction;
        $next = get_next_collision($caret, $direction, \%h_table, \%v_table);

        last unless defined $next;

        # calculate next caret and the points that we crossed
        $to_caret = [ $next->{a} - $direction->[0], $next->{b} - $direction->[1] ];
        push @stepped_points, map { [ @$_, $last_direction ] } point_range($caret, $to_caret);
        warn "break on moving from $caret to $to_caret" if (grep exists $point_table{$_}, map { join ',', @$_ } point_range($caret, $to_caret));

        push @loops_found, test_find_loop($caret, $to_caret, $direction, \%h_table, \%v_table, \@previous_impacts);
        # foreach my $point (point_range($caret, $to_caret)) {
        #     my $next_direction = [ $direction->[1], -$direction->[0] ];
        #     my $test_collision = get_next_collision($point, $next_direction, \%h_table, \%v_table);
        #     next unless defined $test_collision;
        #     # say "found wat:", Dumper $test_collision;
        #     my @matches = grep {
        #         $test_collision->{a} == $_->[0]{a}
        #         && $test_collision->{b} == $_->[0]{b}
        #         && $next_direction->[0] == $_->[1][0]
        #         && $next_direction->[1] == $_->[1][1]} @previous_impacts;
        #     if (@matches > 0) {
        #         say "found match:", Dumper \@matches;
        #     }
        # }

        push @previous_impacts, [ $next, $direction ];
        # update state
        $caret = $to_caret;
        $direction = [ $direction->[1], -$direction->[0] ];
    }

    # no further collisions, we go all the way to the wall
    $to_caret = [
        ($caret->[0], $shape->[0]-1, 0)[$direction->[0]],
        ($caret->[1], $shape->[1]-1, 0)[$direction->[1]],
    ];
    push @loops_found, test_find_loop($caret, $to_caret, $direction, \%h_table, \%v_table, \@previous_impacts);
    push @stepped_points, map { [ @$_, $last_direction ] } point_range($caret, $to_caret);
    warn "break on moving from $caret to $to_caret" if (grep exists $point_table{$_}, map { join ',', @$_ } point_range($caret, $to_caret));
    # \%point_table;
    # @previous_impacts;
    # @stepped_points;
    # sum map 1, keys %stepped_points;
    # $shape;
    # $to_caret;

    @loops_found;
}

sub process_day6_part2 {
    sum map 1, process_day6_part2_get_stepped(@_);
}
# 0 < 4
# 7 < 4

# 0*-1 < 4*-1 : 0 < -4
# 7*-1 < 4*-1 : -7 < -4

# say Dumper get_caret($practice_input);
# say Dumper process_day6_part2_get_stepped($practice_input);
say Dumper process_day6_part2($practice_input);
# say Dumper grep @$_ > 1, values %{{ group process_day6_part2_get_stepped($practice_input) }};

$VAR1 = 6;



1

In [11]:
sub get_points {
    my ($arr) = @_;
    my @points;
    for my $a (0 .. $#$arr) {
        push @points, map { { a => $a, b => $_ } } grep $arr->[$a][$_] eq '#', 0 .. $#{$arr->[0]};
    }
    return @points;
}
sub get_caret_from_arr {
    my ($arr) = @_;
    my @points;
    for my $a (0 .. $#$arr) {
        push @points, map { [ $a, $_ ] } grep $arr->[$a][$_] eq '^', 0 .. $#{$arr->[0]};
    }
    return $points[0];
}
sub recombine_map {
    my ($arr) = @_;
    return join "\n", map { join '', @$_ } @$arr;
}

sub simulate_walk {
    my ($input, $caret, $direction) = @_;

    my $arr = parse_2d_map_array($input);
    $caret //= get_caret_from_arr($arr);
    $direction //= [-1,0];
    my @points = get_points($arr);
    # my @points =
    #     grep defined,
    #     flatten_nd
    #     map_nd selector(qq/where.pos/),
    #     map_nd_indexed graphql_query(qq/{ where(value:'#') {
    #         pos
    #     }}/),
    #     parse_2d_map_array($input);
    my %point_table = map { ($_->{a} . ',' . $_->{b}) => 1 } @points;
    my $shape = shape_nd($arr);

    my %v_table = group_by('b', @points);
    my %h_table = group_by('a', @points);


    my $next = 1;
    my $to_caret;
    my @stepped_points;
    my $impact_id;

    my %previous_impacts;

    while (defined $next) {
        $next = get_next_collision($caret, $direction, \%h_table, \%v_table);

        # calculate next caret and the points that we crossed
        if (defined $next) {
            $impact_id = join ',', @{$next}{qw/ a b /}, @$direction;
            $to_caret = [ $next->{a} - $direction->[0], $next->{b} - $direction->[1] ];
        } else {
            $impact_id = '';
            $to_caret = [
                ($caret->[0], $shape->[0]-1, 0)[$direction->[0]],
                ($caret->[1], $shape->[1]-1, 0)[$direction->[1]],
            ];
        }
        push @stepped_points, map { join ',', @$_ } point_range($caret, $to_caret);
        warn "break on moving from $caret to $to_caret" if (grep exists $point_table{$_}, map { join ',', @$_ } point_range($caret, $to_caret));

        # update state
        $caret = $to_caret;
        $direction = [ $direction->[1], -$direction->[0] ];

        # break out of loop if we have looped!
        if (exists $previous_impacts{$impact_id}) {
            push @stepped_points, 'loop';
            last;
        }
        # otherwise record this event
        $previous_impacts{$impact_id} = 1;
    }

    # @loops_found;
    @stepped_points;
}

In [22]:
sub test_find_loop {
    my ($from, $to, $direction, $input) = @_;
    my $next_direction = [ $direction->[1], -$direction->[0] ];
    my @loops_found;

    my $shape = shape_nd(parse_2d_map_array($input));
    foreach my $point (point_range($from, $to)) {
        my $next_input = parse_2d_map_array($input);
        my $next_obstacle = [ $point->[0] + $direction->[0], $point->[1] + $direction->[1] ];
        next unless ($next_obstacle->[0] >= 0
                and $next_obstacle->[1] >= 0
                and $next_obstacle->[0] < $shape->[0]
                and $next_obstacle->[1] < $shape->[0]);
        next unless $next_input->[$next_obstacle->[0]][$next_obstacle->[1]] ne '#';
        $next_input->[$next_obstacle->[0]][$next_obstacle->[1]] = '#';
        $next_input = recombine_map($next_input);
        my @res = simulate_walk($next_input, $point, $next_direction);
         if (@res and $res[-1] eq 'loop') {
             push @loops_found, $next_obstacle->[0] . ',' . $next_obstacle->[1];
         }
        # say ("loop found!\n", $next_input) if @res and $res[-1] eq 'loop';
        # say "testing:\n", $next_input;
    }
    return @loops_found;
}

Warning: Subroutine test_find_loop redefined at reply input line 1.


In [31]:





sub simulate_walk_with_loops {
    my ($input) = @_;

    my $arr = parse_2d_map_array($input);
    my $caret = get_caret_from_arr($arr);
    my $direction = [-1,0];
    my @points = get_points($arr);
    # my @points =
    #     grep defined,
    #     flatten_nd
    #     map_nd selector(qq/where.pos/),
    #     map_nd_indexed graphql_query(qq/{ where(value:'#') {
    #         pos
    #     }}/),
    #     parse_2d_map_array($input);
    my %point_table = map { ($_->{a} . ',' . $_->{b}) => 1 } @points;
    my $shape = shape_nd($arr);

    my %v_table = group_by('b', @points);
    my %h_table = group_by('a', @points);

    my $next = 1;
    my $to_caret;
    my @stepped_points;
    my $impact_id;

    my %previous_impacts;
    my @loops_found;

    while (defined $next) {
        $next = get_next_collision($caret, $direction, \%h_table, \%v_table);

        # calculate next caret and the points that we crossed
        if (defined $next) {
            $impact_id = join ',', @{$next}{qw/ a b /}, @$direction;
            $to_caret = [ $next->{a} - $direction->[0], $next->{b} - $direction->[1] ];
        } else {
            $impact_id = '';
            $to_caret = [
                ($caret->[0], $shape->[0]-1, 0)[$direction->[0]],
                ($caret->[1], $shape->[1]-1, 0)[$direction->[1]],
            ];
        }
        push @stepped_points, map { join ',', @$_ } point_range($caret, $to_caret);
        
        warn "break on moving from $caret to $to_caret" if (grep exists $point_table{$_}, map { join ',', @$_ } point_range($caret, $to_caret));
        push @loops_found, test_find_loop($caret, $to_caret, $direction, $input);

        # update state
        $caret = $to_caret;
        $direction = [ $direction->[1], -$direction->[0] ];

        # break out of loop if we have looped!
        if (exists $previous_impacts{$impact_id}) {
            push @stepped_points, 'loop';
            last;
        }
        # otherwise record this event
        $previous_impacts{$impact_id} = 1;
    }

    \@stepped_points,
    \@loops_found;
}

my $practice_input = qq/
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
/;

sub uniq { my %h; @h{@_} = (); keys %h }
# sub process_day6_part2 { sum map 1, uniq simulate_walk_with_loops(@_) }
# say Dumper simulate_walk_with_loops($practice_input);


sub update_map_path2 {
    my ($map, $shape, $path, $blocks) = @_;
    foreach my $point (map [ split ',' ], @$path) {
        # say 'wat:', $map->[$point->[0] * $shape->[1] + $point->[1]];
        $map->[$point->[0] * $shape->[1] + $point->[1]] = $map->[$point->[0] * $shape->[1] + $point->[1]] eq '#' ? '!' : 'X';
    }
    foreach my $point (map [ split ',' ], @$blocks) {
        # say 'wat:', $map->[$point->[0] * $shape->[1] + $point->[1]];
        $map->[$point->[0] * $shape->[1] + $point->[1]] = $map->[$point->[0] * $shape->[1] + $point->[1]] eq '#' ? '?' : '%';
    }
}
sub string_map {
    my ($map, $shape) = @_;
    join "\n", map { join '', @{$map}[$_ * $shape->[1] .. (($_+1) * $shape->[1]-1)] } 0 .. $shape->[0] - 1;
}
my $data = parse_2d_map_array($practice_input);
my $shape = shape_nd($data);
my @map = flatten_nd $data;
update_map_path2(\@map, $shape, simulate_walk_with_loops($practice_input));
say string_map(\@map, $shape);



# my $data = parse_2d_map_array($practice_input);
# my $shape = shape_nd($data);
# my @map = flatten_nd $data;
# update_map_path(\@map, $shape, [ grep $_ ne 'loop', simulate_walk($practice_input) ]);
# say string_map(\@map, $shape);

....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#X%XXXXX.
.XXXXX%%#.
#%X%XXXX..
......#%..


1

Warning: Subroutine simulate_walk_with_loops redefined at reply input line 6.

Subroutine uniq redefined at reply input line 84.

Subroutine update_map_path2 redefined at reply input line 89.

Subroutine string_map redefined at reply input line 100.


In [32]:
my $input = get_challenge('2024/day/6/input');
my $data = parse_2d_map_array($input);
my $shape = shape_nd($data);
my @map = flatten_nd $data;
update_map_path2(\@map, $shape, simulate_walk_with_loops($input));
say string_map(\@map, $shape);
writefile('.exp/testmap', string_map(\@map, $shape));

...#....X...................................#........#.................#............#.#.......................#...............#..#
...#....%...#.....#..................................................................................#......#..#..................
........X........#.......................#.#........#......#...........................................#...#..........#...........
........%.........#..................#...#.....................................#........#.........................................
...#....%#.......#..............#.#....##.............#....##......................................#...................#..........
...#....XX%%XXXXXXXXX%XXXXXXXXXXXXXXXX%XXXXXXX%%X%XXXXX%%XXXXXXXXXXXXXX%X%X%%X%XXXX%XXXXXXX%XXXXXXXXXXX%%X%XX%XXX%XXXXX%XXXXXX#...
..#..#..%X.........#.....#...............#.......#..............................................#....#.......................X....
........%X.........%%%#..#...............#...................#....#................

1

In [26]:
my $input = get_challenge('2024/day/6/input');
my $res = process_day6_part2($input);

2088

In [27]:
# more than 500, less than 2258
say post_answer('2024/day/6/answer', 2, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 6 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're 

1